In [ ]:
import os, threading

os.environ["GOLD_DB_PATH"] = "gold_prices.sqlite"
os.environ["GOLD_CURRENCY"] = "USD"
os.environ["GOLD_DB_POLL_SEC"] = "15"
os.environ["GOLD_WINDOW_N"] = "288"          # 24 uur bij 5m
os.environ["GOLD_BREAK_BUFFER_PCT"] = "0.01" # 1% buffer
os.environ["GOLD_PUSH_MODE"] = "signal"      # alleen ENTER meldingen

t = threading.Thread(target=run_forever, daemon=True)
t.start()



[watch] DB=gold_prices.sqlite table=gold_spot_prices currency=USD poll=10s state=gold_watch_state.json


In [ ]:
import os
import time
import json
import sqlite3
import xml.etree.ElementTree as ET
from typing import Any, Dict, List, Optional, Tuple

import requests

# =========================
# Config (5m-friendly defaults)
# =========================
DB_PATH = os.getenv("GOLD_DB_PATH", "gold_prices.sqlite")
TABLE = os.getenv("GOLD_TABLE", "gold_spot_prices")
CURRENCY = os.getenv("GOLD_CURRENCY", "USD").upper()

# DB polling (hoe vaak kijken of er een nieuwe row is)
POLL_SEC = int(os.getenv("GOLD_DB_POLL_SEC", "15"))

# 5m sampling: 288 = 24 uur, 144 = 12 uur, 72 = 6 uur
WINDOW_N = int(os.getenv("GOLD_WINDOW_N", "288"))

# Break buffer (percentage van range) – 5m: 1% is vaak genoeg om ruis te filteren
BREAK_BUFFER_PCT = float(os.getenv("GOLD_BREAK_BUFFER_PCT", "0.01"))

# TP projecties (multiples van range)
TP1_MULT = float(os.getenv("GOLD_TP1_MULT", "0.6"))
TP2_MULT = float(os.getenv("GOLD_TP2_MULT", "1.2"))

# Afronding van levels (0 = hele dollars)
LEVEL_ROUND_DECIMALS = int(os.getenv("GOLD_LEVEL_ROUND_DECIMALS", "0"))

# Push gedrag:
# - "signal" = alleen push bij ENTER LONG/SHORT (aanrader)
# - "always" = bij elke nieuwe row push met levels
PUSH_MODE = os.getenv("GOLD_PUSH_MODE", "signal").lower()

# Prowl
PROWL_KEY_FILE = os.getenv("PROWL_KEY_FILE", "key.txt")
PROWL_APPLICATION = os.getenv("PROWL_APPLICATION", "Gold Setups (DB 5m)")
PROWL_PRIORITY = int(os.getenv("PROWL_PRIORITY", "0"))  # -2..2
PROWL_TIMEOUT = float(os.getenv("PROWL_TIMEOUT_SEC", "20"))
USER_AGENT = os.getenv("GOLD_USER_AGENT", "gold-db-setups/5m")

# State
STATE_FILE = os.getenv("GOLD_WATCH_STATE_FILE", "gold_setups_state.json")


# =========================
# Prowl
# =========================
def read_prowl_key(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        key = f.read().strip()
    if not key:
        raise RuntimeError(f"Prowl key file is empty: {path}")
    return key


def prowl_send(apikey: str, application: str, event: str, description: str, priority: int = 0) -> None:
    url = "https://api.prowlapp.com/publicapi/add"
    payload = {
        "apikey": apikey,
        "application": application,
        "event": event,
        "description": description,
        "priority": str(priority),
    }
    r = requests.post(url, data=payload, timeout=PROWL_TIMEOUT, headers={"User-Agent": USER_AGENT})

    if r.status_code == 406:
        raise RuntimeError("Prowl API limit exceeded (HTTP 406)")
    if r.status_code != 200:
        raise RuntimeError(f"Prowl HTTP {r.status_code}: {r.text[:200]}")

    try:
        root = ET.fromstring(r.text)
        err = root.find("error")
        if err is not None:
            code = err.attrib.get("code", "?")
            msg = (err.text or "").strip()
            raise RuntimeError(f"Prowl error {code}: {msg}")
    except ET.ParseError:
        pass


# =========================
# State
# =========================
def load_state() -> Dict[str, Any]:
    try:
        with open(STATE_FILE, "r", encoding="utf-8") as f:
            return json.load(f)
    except FileNotFoundError:
        return {}
    except Exception:
        return {}


def save_state(state: Dict[str, Any]) -> None:
    with open(STATE_FILE, "w", encoding="utf-8") as f:
        json.dump(state, f, ensure_ascii=False, indent=2)


# =========================
# DB (read-only)
# =========================
def fetch_latest_rows(db_path: str, table: str, currency: str, n: int) -> List[Dict[str, Any]]:
    q = f"""
    SELECT id, ts_utc, source, currency, price_per_troy_oz
    FROM {table}
    WHERE currency = ?
    ORDER BY id DESC
    LIMIT ?
    """
    with sqlite3.connect(db_path) as con:
        rows = con.execute(q, (currency, n)).fetchall()

    out: List[Dict[str, Any]] = []
    for r in rows:
        out.append({
            "id": int(r[0]),
            "ts_utc": str(r[1]),
            "source": str(r[2]),
            "currency": str(r[3]),
            "price": float(r[4]),
        })
    return out


# =========================
# Setup calculation
# =========================
def rr(x: float) -> float:
    return round(x, LEVEL_ROUND_DECIMALS)


def compute_setups(rows_newest_first: List[Dict[str, Any]]) -> Dict[str, Any]:
    if len(rows_newest_first) < 20:
        raise RuntimeError("Te weinig rows in DB om 5m setups stabiel te berekenen (minimaal 20).")

    latest = rows_newest_first[0]
    prices = [x["price"] for x in rows_newest_first]

    hi = max(prices)
    lo = min(prices)
    rng = max(hi - lo, 1e-9)
    mid = (hi + lo) / 2.0

    buf = rng * BREAK_BUFFER_PCT

    # Triggers: boven dag-high + buffer, onder dag-low - buffer
    long_trigger = hi + buf
    short_trigger = lo - buf

    # SL template: mid (range invalidation)
    sl = mid

    # TP templates: projecties vanaf range-extremes
    tp1_long = hi + rng * TP1_MULT
    tp2_long = hi + rng * TP2_MULT

    tp1_short = lo - rng * TP1_MULT
    tp2_short = lo - rng * TP2_MULT

    current = latest["price"]

    if current >= long_trigger:
        signal = "ENTER_LONG"
    elif current <= short_trigger:
        signal = "ENTER_SHORT"
    else:
        signal = "WAIT"

    return {
        "ts_utc": latest["ts_utc"],
        "source": latest["source"],
        "current": current,
        "hi": hi,
        "lo": lo,
        "range": rng,
        "mid": mid,
        "buf": buf,
        "long_trigger": long_trigger,
        "short_trigger": short_trigger,
        "sl": sl,
        "tp1_long": tp1_long,
        "tp2_long": tp2_long,
        "tp1_short": tp1_short,
        "tp2_short": tp2_short,
        "signal": signal,
    }


def format_signal_message(setup: Dict[str, Any]) -> Tuple[str, str, str]:
    cur = setup["current"]

    long_tr = setup["long_trigger"]
    short_tr = setup["short_trigger"]
    sl = setup["sl"]

    # Minimal output: alleen setup-waarden
    if setup["signal"] == "ENTER_LONG":
        event = f"ENTER LONG | Gold {CURRENCY} {rr(cur):.0f}"
        body = "\n".join([
            f"Prijs: {rr(cur):.0f}",
            "",
            f"LONG trigger:  >= {rr(long_tr):.0f}",
            f"SL: {rr(sl):.0f}",
            f"TP: {rr(setup['tp1_long']):.0f} → {rr(setup['tp2_long']):.0f}",
            "",
            f"Context (window {WINDOW_N}x5m): Hi={rr(setup['hi']):.0f} Lo={rr(setup['lo']):.0f} Range={rr(setup['range']):.0f}",
            f"Bron: {setup['source']} | Tijd (UTC): {setup['ts_utc']}",
        ])
        regime_key = f"ENTER_LONG|{rr(long_tr)}|{rr(short_tr)}"
        return event, body, regime_key

    if setup["signal"] == "ENTER_SHORT":
        event = f"ENTER SHORT | Gold {CURRENCY} {rr(cur):.0f}"
        body = "\n".join([
            f"Prijs: {rr(cur):.0f}",
            "",
            f"SHORT trigger: <= {rr(short_tr):.0f}",
            f"SL: {rr(sl):.0f}",
            f"TP: {rr(setup['tp1_short']):.0f} → {rr(setup['tp2_short']):.0f}",
            "",
            f"Context (window {WINDOW_N}x5m): Hi={rr(setup['hi']):.0f} Lo={rr(setup['lo']):.0f} Range={rr(setup['range']):.0f}",
            f"Bron: {setup['source']} | Tijd (UTC): {setup['ts_utc']}",
        ])
        regime_key = f"ENTER_SHORT|{rr(long_tr)}|{rr(short_tr)}"
        return event, body, regime_key

    # WAIT message (only if PUSH_MODE=always)
    event = f"Levels | Gold {CURRENCY} {rr(cur):.0f}"
    body = "\n".join([
        f"Prijs: {rr(cur):.0f}",
        "",
        "WAIT (geen entry)",
        f"LONG trigger:  >= {rr(long_tr):.0f}",
        f"SHORT trigger: <= {rr(short_tr):.0f}",
        f"SL (template): {rr(sl):.0f}",
        "",
        f"Context (window {WINDOW_N}x5m): Hi={rr(setup['hi']):.0f} Lo={rr(setup['lo']):.0f} Range={rr(setup['range']):.0f}",
        f"Bron: {setup['source']} | Tijd (UTC): {setup['ts_utc']}",
    ])
    regime_key = f"WAIT|{rr(long_tr)}|{rr(short_tr)}"
    return event, body, regime_key


# =========================
# Watch loop
# =========================
def run_forever() -> None:
    apikey = read_prowl_key(PROWL_KEY_FILE)
    state = load_state()

    last_seen_id = state.get("last_seen_id")
    last_regime_key = state.get("last_regime_key")

    print(f"[watch] 5m DB={DB_PATH} table={TABLE} currency={CURRENCY} poll={POLL_SEC}s window={WINDOW_N} push={PUSH_MODE}")

    while True:
        try:
            rows = fetch_latest_rows(DB_PATH, TABLE, CURRENCY, WINDOW_N)
            if not rows:
                time.sleep(POLL_SEC)
                continue

            latest_id = rows[0]["id"]

            # First run: set cursor, no push
            if last_seen_id is None:
                last_seen_id = latest_id
                state["last_seen_id"] = last_seen_id
                save_state(state)
                time.sleep(POLL_SEC)
                continue

            if latest_id > int(last_seen_id):
                setup = compute_setups(rows)
                event, body, regime_key = format_signal_message(setup)

                should_push = (PUSH_MODE == "always") or (setup["signal"] in ("ENTER_LONG", "ENTER_SHORT"))

                if should_push and regime_key != last_regime_key:
                    prowl_send(apikey, PROWL_APPLICATION, event, body, PROWL_PRIORITY)
                    last_regime_key = regime_key
                    state["last_regime_key"] = last_regime_key

                last_seen_id = latest_id
                state["last_seen_id"] = last_seen_id
                save_state(state)

        except Exception as e:
            print("[watch][ERROR]", str(e)[:400])

        time.sleep(POLL_SEC)


if __name__ == "__main__":
    run_forever()


[watch] 5m DB=gold_prices.sqlite table=gold_spot_prices currency=USD poll=5s window=120 push=signal
# als we geen 'prev' hebben, geen vergelijking mogelijk
4347.100098 4347.100098
# als we geen 'prev' hebben, geen vergelijking mogelijk
4349.700195 4347.100098
gestegen
# als we geen 'prev' hebben, geen vergelijking mogelijk
4349.700195 4349.700195
# als we geen 'prev' hebben, geen vergelijking mogelijk
4351.100098 4349.700195
gestegen
# als we geen 'prev' hebben, geen vergelijking mogelijk
4353.299805 4351.100098
gestegen
# als we geen 'prev' hebben, geen vergelijking mogelijk
4352.200195 4353.299805
gedaald
# als we geen 'prev' hebben, geen vergelijking mogelijk
4352.200195 4352.200195
# als we geen 'prev' hebben, geen vergelijking mogelijk
4348.200195 4352.200195
gedaald
# als we geen 'prev' hebben, geen vergelijking mogelijk
4348.200195 4348.200195
# als we geen 'prev' hebben, geen vergelijking mogelijk
4348.0 4348.200195
gedaald
# als we geen 'prev' hebben, geen vergelijking mogelij